# 正则表达式样例解析

## 适用正则表达式的场景
我们日常工作或生活，都会遇到大量的文本信息，往往需要做文本查找。<br>
对于简单的查询，比如查找"Close"这个文字在哪，通过ctrl+F即可做到简单查询。<br>
但是或许我们有如下的一些需求，比如获取文档中的日期、数字，包含某个或某些关键字的一段话。<br>
这种情况用简单的查询，就很难定位到所有相关的文本信息。<br>
那么正则表达式，就可以华丽登场了~~~

### 查找SEC文档中的日期
SEC文章中的格式，往往是这种格式：英文单词月 数字日1~2位, 数字年4位
那么正则表达式可以这样写：

In [8]:
import re
def getlistiterbyregex(keyword, text):
    pattern = re.compile(keyword, re.I)
    return re.finditer(pattern, text)

text = r'The effectivedate is May 1, 2018, the disclosed date is April 25, 2018. '
keyword = r'(january|february|march|april|may|june|july|august|september|october|november|december)[\s]*[0-9]{1,2}[\s]*,[\s]*[0-9]{4}'
matchlist = getlistiterbyregex(keyword, text)
for match in matchlist:
    print(match.group(0))

May 1, 2018
April 25, 2018


### 查找文档中的含有百分号的数字

In [10]:
text = r'''Additional Taxes Payable On Withdrawals, Surrenders, Or Annuity Payouts
The Code may impose a 10% additional tax on any distribution from your contract which you must include in your gross income. The 10% additional tax does not apply if one of several exceptions exists. These exceptions include withdrawals, surrenders, or Annuity Payouts that:
you receive on or after you reach 59 , you receive because you became disabled (as defined in the Code), you receive from an immediate annuity, a Beneficiary receives on or after your death, or you receive as a series of substantially equal periodic payments based on your life or life expectancy (non-natural owners holding as agent for an individual do not qualify).
Unearned Income Medicare Contribution
Congress enacted the “Unearned Income Medicare Contribution” as a part of the Health Care and Education Reconciliation Act of 2010. This tax, which affects individuals whose modified adjusted gross income exceeds certain thresholds, is a 3.8% tax on the lesser of (i) the individual's “unearned income”, or (ii) the dollar amount by which the individual's modified adjusted gross income exceeds the applicable threshold. Unearned income includes the taxable portion of distributions that you take from your annuity contract. If you take a distribution from your contract that may be subject to the tax, we will include a Distribution Code “D” in Box 7 of the Form 1099-R issued to report the distribution. Please consult your tax advisor to determine whether your annuity distributions are subject to this tax.
Special Rules If You Own More Than One Annuity Contract'''
keyword = r'[\d]*(\.[\d]*)?\%'
matchlist = getlistiterbyregex(keyword, text)
for match in matchlist:
    print(match.group(0))

10%
10%
3.8%


### 根据分组从一段话中拿到特定内容
分组，在正则表达式是通过括号:\(\)实现的，如果加入```(?<groupname>正则表达式)```，即可实现有名称的分组。<br>
但是如果用python，则需要这样写:```(?P<groupname>正则表达式)```

In [19]:
text = r'The JPMorgan Insurance Trust Intrepid Mid Cap Portfolio intends to liquidate.'
keyword = 'The(?P<sharename>(.*)?)intends to liquidate'
matchlist = getlistiterbyregex(keyword, text)
groupname = 'sharename'
groupnames = []
for match in matchlist:
    groupnames = match.groupdict().keys() if len(groupnames) == 0 else groupname
    sharename = match.group(groupname) if groupname in groupnames else ''
    print(sharename)
    

 JPMorgan Insurance Trust Intrepid Mid Cap Portfolio 


### 通过或的方式达到匹配多种情况
有时一段话的关键字，往往有多种时态，比如过去时，现在进行时等，那么只写一种情况，或许无法匹配多种情况。<br>
可以通过或的方式解决这种问题。<br>
如下面的例子，是为了获取merge相关的previous与after share name。<br>
这个例子还可以达到演示通过多个分组，分别获取相应信息的目的。

In [24]:
text = r'''On March 15, 2018, the Board of Trustees of Voya Investors Trust approved a proposal to reorganize the Voya Multi-Manager Large Cap Core Portfolio (the “Reorganization”). 
Subject to shareholder approval, effective after the close of business on or about August 24, 2018 (the “Reorganization Date”), 
Class I shares of the Voya Multi-Manager Large Cap Core Portfolio (the “Merging Fund”) will be merged into 
Class I shares of the Voya Index Plus LargeCap Portfolio (the “Surviving Fund”).'''
keyword = 'on or about[\s]*?(?P<pendingdate>(january|february|march|april|may|june|july|august|september|october|november|december)[\s]*[0-9]{1,2}[\s]*,[\s]*[0-9]{4})[\s]*?\((.*)?\),[\s]*?(?P<previous>(.*)?)\(the(.*)?fund(”)?\)[\s]*?(.*)?merg(e|ed|ing) into[\s]*?(?P<after>(.*)?)\(the(.*)?fund(”)?\)\.'
matchlist = getlistiterbyregex(keyword, text)

groupnames = ['pendingdate', 'previous', 'after']
for match in matchlist:
    groupnamelist = match.groupdict().keys() if len(groupnames) == 0 else groupname
    for groupname in groupnames:
        print('{0} is {1}'.format(groupname, match.group(groupname) if groupname in groupnames else ''))

pendingdate is August 24, 2018
previous is Class I shares of the Voya Multi-Manager Large Cap Core Portfolio 
after is Class I shares of the Voya Index Plus LargeCap Portfolio 


### 先拿段落再拿明细的正则表达式写法
有时候，我们会遇到更复杂的文档。<br>
比如确定share name的段落，有明确的标识：UNDERLYING FUNDS:，但是share name是一个列表。<br>
这个时候，一般的做法，是先将段落，确定下来。<br>
然后再从段落里面，拿share name。<br>
示例：

In [56]:
text = r'''
UNDERLYING FUNDS:                                    MANAGED BY:                                  TRUST
--------------------------------------------------   ------------------------------------------   -------
                                                                                            
 Franklin Founding Funds Allocation VIP Fund         Franklin Templeton Services, LLC             FTVIPT
 Franklin Income VIP Fund                            Franklin Advisers, Inc.                      FTVIPT
 SA Allocation Balanced Portfolio                    SunAmerica Asset Management, LLC             SST
 SA Allocation Growth Portfolio                      SunAmerica Asset Management, LLC             SST
 SA Allocation Moderate Portfolio                    SunAmerica Asset Management, LLC             SST
 SA Allocation Moderate Growth Portfolio             SunAmerica Asset Management, LLC             SST
 SA American Funds(R) Asset Allocation Portfolio     Capital Research and Management Company      SAST
 SA BlackRock Multi-Asset Income Portfolio           BlackRock Investment Management, LLC         AST
 SA Edge Asset Allocation Portfolio                  Principal Global Investors, LLC              AST
 SA JPMorgan Diversified Balanced Portfolio          J.P. Morgan Investment Management Inc.       SAST
 SA MFS Total Return Portfolio*                      Massachusetts Financial Services Company     SAST



'''
keyword = r'[\n]UNDERLYING FUNDS(.*)[\n]-{5,}(.*)?[\n](?P<multiname>[\s\S]*?)[\n][\n]'
matchlist = getlistiterbyregex(keyword, text)
segment = ''
for match in matchlist:
    segment = match.group('multiname')
    print(segment)

                                                                                            
 Franklin Founding Funds Allocation VIP Fund         Franklin Templeton Services, LLC             FTVIPT
 Franklin Income VIP Fund                            Franklin Advisers, Inc.                      FTVIPT
 SA Allocation Balanced Portfolio                    SunAmerica Asset Management, LLC             SST
 SA Allocation Growth Portfolio                      SunAmerica Asset Management, LLC             SST
 SA Allocation Moderate Portfolio                    SunAmerica Asset Management, LLC             SST
 SA Allocation Moderate Growth Portfolio             SunAmerica Asset Management, LLC             SST
 SA American Funds(R) Asset Allocation Portfolio     Capital Research and Management Company      SAST
 SA BlackRock Multi-Asset Income Portfolio           BlackRock Investment Management, LLC         AST
 SA Edge Asset Allocation Portfolio                  Principal Global Investors, LLC

In [57]:
if len(segment) > 0:
    keyword = r'[\n](?P<sharename>(.*)?(fund|portfolio)(\*)?)'
    matchlist = getlistiterbyregex(keyword, segment)
    for index, match in enumerate(matchlist):
        print('share {0}, name is: {1}'.format(index + 1, match.group('sharename')))

share 1, name is:  Franklin Founding Funds Allocation VIP Fund
share 2, name is:  Franklin Income VIP Fund
share 3, name is:  SA Allocation Balanced Portfolio
share 4, name is:  SA Allocation Growth Portfolio
share 5, name is:  SA Allocation Moderate Portfolio
share 6, name is:  SA Allocation Moderate Growth Portfolio
share 7, name is:  SA American Funds(R) Asset Allocation Portfolio
share 8, name is:  SA BlackRock Multi-Asset Income Portfolio
share 9, name is:  SA Edge Asset Allocation Portfolio
share 10, name is:  SA JPMorgan Diversified Balanced Portfolio
share 11, name is:  SA MFS Total Return Portfolio*


另一个示例

In [80]:
text = r'''
Variable Account Options
--------------------------------------------------------------------------------



                                     
    VALIC Company I Funds               VALIC Company II Funds
    Asset Allocation Fund               Aggressive Growth Lifestyle Fund
    Blue Chip Growth Fund               Capital Appreciation Fund
    Broad Cap Value Income Fund         Core Bond Fund
    Capital Conservation Fund           Conservative Growth Lifestyle Fund
    Core Equity Fund                    Government Money Market II Fund
    Dividend Value Fund                 High Yield Bond Fund
    Emerging Economies Fund             International Opportunities Fund
    Foreign Value Fund                  Large Cap Value Fund
    Global Real Estate Fund             Mid Cap Growth Fund
    Global Social Awareness Fund        Mid Cap Value Fund
    Global Strategy Fund                Moderate Growth Lifestyle Fund
    Government Money Market I Fund      Small Cap Growth Fund
    Government Securities Fund          Small Cap Value Fund
    Growth Fund                         Socially Responsible Fund
    Growth & Income Fund                Strategic Bond Fund
    Health Sciences Fund
    Inflation Protected Fund
    International Equities Index Fund
    International Government Bond Fund
    International Growth Fund
    Large Cap Core Fund
    Large Capital Growth Fund
    Mid Cap Index Fund
    Mid Cap Strategic Growth Fund
    Nasdaq-100(R) Index Fund
    Science & Technology Fund
    Small Cap Aggressive Growth Fund
    Small Cap Fund
    Small Cap Index Fund
    Small Cap Special Values Fund
    Small-Mid Growth Fund
    Stock Index Fund
    Value Fund





Table of Contents
--------------------------------------------------------------------------------

'''

In [114]:
keyword = r'[\n]Variable Account Options[\n]-{5,}(.*)?[\n](?P<multiname>[\s\S]*?)[\n][\n]Table of Contents[\n]'
matchlist = getlistiterbyregex(keyword, text)
segmentforname = ''
for match in matchlist:
    segmentforname = match.group('multiname')
    print(segmentforname)




                                     
    VALIC Company I Funds               VALIC Company II Funds
    Asset Allocation Fund               Aggressive Growth Lifestyle Fund
    Blue Chip Growth Fund               Capital Appreciation Fund
    Broad Cap Value Income Fund         Core Bond Fund
    Capital Conservation Fund           Conservative Growth Lifestyle Fund
    Core Equity Fund                    Government Money Market II Fund
    Dividend Value Fund                 High Yield Bond Fund
    Emerging Economies Fund             International Opportunities Fund
    Foreign Value Fund                  Large Cap Value Fund
    Global Real Estate Fund             Mid Cap Growth Fund
    Global Social Awareness Fund        Mid Cap Value Fund
    Global Strategy Fund                Moderate Growth Lifestyle Fund
    Government Money Market I Fund      Small Cap Growth Fund
    Government Securities Fund          Small Cap Value Fund
    Growth Fund                         Sociall

这个例子不仅将share name识别出来，而且将第一行作为share header进行拼接

In [113]:
if len(segmentforname) > 0:
    keyword = '[\n][ ]{2,}(?P<sharename>(.*)?)'
    matchlist = getlistiterbyregex(keyword, segmentforname)
    shareheaders = []
    isfirstline = True
    for match in matchlist:
        shareinfo = match.group('sharename')
        sharelist = [share for share in shareinfo.split('  ') if len(share.strip()) > 0]
        for index, share in enumerate(sharelist):
            if len(share.strip()) > 0:
                if isfirstline:
                    shareheaders.append(share)
                else:
                    sharename = '{0} {1}'.format(shareheaders[index], share)
                    print('sharename is {0}'.format(sharename))
        if len(shareheaders) > 0:
            isfirstline = False

sharename is VALIC Company I Funds Asset Allocation Fund
sharename is  VALIC Company II Funds  Aggressive Growth Lifestyle Fund
sharename is VALIC Company I Funds Blue Chip Growth Fund
sharename is  VALIC Company II Funds  Capital Appreciation Fund
sharename is VALIC Company I Funds Broad Cap Value Income Fund
sharename is  VALIC Company II Funds  Core Bond Fund
sharename is VALIC Company I Funds Capital Conservation Fund
sharename is  VALIC Company II Funds  Conservative Growth Lifestyle Fund
sharename is VALIC Company I Funds Core Equity Fund
sharename is  VALIC Company II Funds Government Money Market II Fund
sharename is VALIC Company I Funds Dividend Value Fund
sharename is  VALIC Company II Funds  High Yield Bond Fund
sharename is VALIC Company I Funds Emerging Economies Fund
sharename is  VALIC Company II Funds  International Opportunities Fund
sharename is VALIC Company I Funds Foreign Value Fund
sharename is  VALIC Company II Funds Large Cap Value Fund
sharename is VALIC Compa